In [ ]:
import json
from pathlib import Path
import pickle

import numpy as np

from astropy.table import Table

#from try_json_editing_rev1 import whole_thing, generate_json_file_name, exotic_arguments, get_values_from_widget
from stellarphot.transit_fitting.gui import generate_json_file_name, exotic_arguments, get_values_from_widget, exotic_settings_widget, populate_TOI_boxes
from stellarphot.settings.fits_opener import FitsOpener
from stellarphot import PhotometryData
from stellarphot.io import TOI

# Instructions and Overview

This notebook will do a different kind of fit to your exoplanet data. It takes longer to run than the fitting we've done so far, which is why we didn't begin with this type of fit.

In [ ]:
fits_opener = FitsOpener(title="Select your photometry/relative flux file", filter_pattern=["*.ecsv"])

fits_opener.file_chooser

In [ ]:
input_data_file = fits_opener.path
toi_info_file = Path("TIC-194461202-info.json")

tess_info = TOI.model_validate_json(toi_info_file.read_text())

In [ ]:
settings = exotic_settings_widget()

exotic_data_file = 'data_for_exotic.csv'

In [ ]:
inp_data = PhotometryData.read(input_data_file)
inp_data = inp_data[inp_data['star_id'] == 1]
inp_data.add_bjd_col()
# inp_data = inp_data[inp_data['BJD'] > 2459795.75]

out_data = inp_data['bjd', 'relative_flux', 'relative_flux_error', 'airmass']
out_data.write(exotic_data_file, overwrite=True)

In [ ]:
settings.data_file_widget['candidate'].value = exotic_data_file
settings.data_file_widget['known'].value = exotic_data_file
populate_TOI_boxes(tess_info, settings.value_widget)

In [ ]:
settings

In [ ]:
settings_name = generate_json_file_name(settings)
contents = get_values_from_widget(settings)

In [ ]:
Path(contents['user_info']['Directory to Save Plots']).mkdir(exist_ok=True)

In [ ]:
with open(settings_name, 'w') as f:
    json.dump(contents, f)

In [ ]:
print('COPY AND PASTE THE LINE BELOW INTO AN EMPTY CELL TO RUN EXOTIC (include the exclamation point at the beginning):\n\n')
print(f'!exotic {" ".join(exotic_arguments[settings.planet_type.value])} {settings_name}')
print("\n\n")

In [ ]:
!exotic --override --pre TIC_194461202-2024-10-08-SR.json